In [120]:
import pandas as pd
import numpy as np
import altair as alt
import json
import nx_altair as nxa
import networkx as nx

data = pd.read_json("tweets.json")
data.head()

retweeted                                             source  \
0      False  <a href="https://mobile.twitter.com" rel="nofo...   
1      False  <a href="https://mobile.twitter.com" rel="nofo...   
2      False  <a href="http://twitter.com/download/android" ...   
3      False  <a href="http://twitter.com/download/android" ...   
4      False  <a href="http://twitter.com/download/android" ...   

                                            entities display_text_range  \
0  {'user_mentions': [{'name': 'D3.js', 'screen_n...           [0, 293]   
1  {'hashtags': [], 'symbols': [], 'user_mentions...           [0, 131]   
2  {'hashtags': [], 'symbols': [], 'user_mentions...           [0, 182]   
3  {'hashtags': [], 'symbols': [], 'user_mentions...           [0, 152]   
4  {'hashtags': [{'text': 'dataviz', 'indices': [...           [0, 122]   

   favorite_count               id_str  truncated  retweet_count  \
0              12  1302979067080302592      False              0   
1               1  1302971932736880640      False              0   
2               2  1302946877386694656      False              0   
3               0  1301867586255228928      False              0   
4               3  1301865933737275392      False              0   

                    id  possibly_sensitive  ...  \
0  1302979067080302592                 0.0  ...   
1  1302971932736880640                 0.0  ...   
2  1302946877386694656                 0.0  ...   
3  1301867586255228928                 NaN  ...   
4  1301865933737275392                 0.0  ...   

                                   extended_entities  hour selected __seqId  \
0  {'media': [{'expanded_url': 'https://twitter.c...     7     True       0   
1                                                NaN     7     True       1   
2                                                NaN     5     True       2   
3                                                NaN     5     True       3   
4                                                NaN     5     True       4   

   __i  in_reply_to_status_id_str  in_reply_to_user_id  in_reply_to_status_id  \
0  [0]                        NaN                  NaN                    NaN   
1  [1]               1.302970e+18         4.127315e+09           1.302970e+18   
2  [2]                        NaN                  NaN                    NaN   
3  [3]               1.301865e+18         3.687505e+08           1.301865e+18   
4  [4]                        NaN                  NaN                    NaN   

  in_reply_to_screen_name  in_reply_to_user_id_str  
0                     NaN                      NaN  
1               markiaaan             4.127315e+09  
2                     NaN                      NaN  
3           jamesheathers             3.687505e+08  
4                     NaN                      NaN  

[5 rows x 24 columns]

In [121]:
# Calculating Edges and Nodes for a network Chart
# Edges are links between hashtags. 
# Nodes are Hashtags


# You need to identify each unique combination of hashtags and nodes and combine them. 
# Hashtags are in the entities column. Make a list of entities values.
# Cast the entities as a dictionary
# Extract the hastags
# Build a list of unique hashtag combos and time. 

list_entities = pd.Series([i['hashtags'] for i in data['entities']])
list_len_bool = [len(i) > 0  for i in list_entities]
list_with_hashtags = list_entities[list_len_bool]
texts = [text[0]['text'] for text in list_with_hashtags]



In [122]:
list_of_hashtags = []
for text in list_with_hashtags:
    output_list = []
    for item in text:
        output_list.append(item['text'])
    list_of_hashtags.append(output_list)
    
# Need to sort each hashtag list alphabetically to ensure that the tuple pairs later are always ordered the same.
for hashtag_set in list_of_hashtags:
    hashtag_set.sort()


In [123]:
def edges_dictionary (hashtag_list):
    '''Input a list of lists with hashtags, this will return a dictionary with tuples of hashtag
    pairs that were used together, and how many times they occurred. 
    '''
    return_dic = {}
    
    for hashtag_set in hashtag_list:
        tuples_list = []
        
        if len(hashtag_set) == 1:
            continue # tuples_list.append( (hashtag_set[0], np.NaN) )
        else:
            for i in range(len(hashtag_set)):
                hashtag = hashtag_set[i]
                if i == len(hashtag_set) - 1:
                    continue #tuples_list.append( (hashtag, np.NaN) )
                else:
                    for g in range(i+1,len(hashtag_set)):
                        tuples_list.append( (hashtag, hashtag_set[g]) )
    
    
        for pairing in tuples_list:
            if pairing in return_dic.keys():
                return_dic[pairing] += 1
            else:
                return_dic[pairing] = 1
    
    return return_dic
    

In [124]:
edges = edges_dictionary(list_of_hashtags)

nodes_list = []
for hashtags in list_of_hashtags:
    for hashtag in hashtags:
        nodes_list.append(hashtag)
        
nodes = np.unique(np.array(nodes_list))


In [135]:
G = nx.Graph()

G.add_nodes_from(nodes)
for nodes, value in edges.items():
    G.add_edge(nodes[0], nodes[1], weight = value)

pos = nx.spring_layout(G)

# Draw the graph using Altair
viz = nxa.draw_networkx(
    G, 
    pos=pos,
    width='weight',
    edge_color='black',
    )
viz.interactive()


alt.LayerChart(...)

In [166]:
top = list(edges.values())
top.sort(reverse = True)
top_25 = top[0:25]
top_25_nodes = []

for nodes, value in edges.items():
    if value in top_25:
        top_25_nodes.append(nodes[0])
        top_25_nodes.append(nodes[1])

top_25_nodes = list(np.unique(np.array(top_25_nodes)))

viz = nxa.draw_networkx(
    G, 
    pos=pos,
    width='weight',
    edge_color='black',
    nodelist = top_25_nodes
    )
viz.interactive()
    

alt.LayerChart(...)